In [2]:
from transformers import pipeline
import nltk
# Load summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Your transcript (long text from Whisper)
transcript = """
Iran's supreme leader Ayatollah Ali Khamenei has warned Washington of "irreparable consequences" if the U.S. becomes directly involved in Israel's military campaign against Tehran. It is understood that President Trump is considering joining the attacks on Iran's nuclear sites. Here's our diplomatic correspondent Caroline Hawley. These satellite images show a military base in Tabriz in the northwest of the country, here before. Now a defiant response from Ayatollah Ali Khamenei (his words read out by a presenter on state-run TV). Iranian president For civilians in Tehran, these are "terrifying times", "shops shut", "quiet streets" many people are held up at home, hiding, we're told, in basements and garages many others have already fled the capital, among them Nagis Muhammadi, winner of the Nobel Peace Prize, who spent many years in prison for his death The Hiren is still firing at Israel but not at the same rate as before. But not at the same rate as before: overnight missiles were intercepted over Tel Aviv, some limited damage reported, but no more casualties. In Cyprus two repatriation flights were laid on to bring Israeli stranded abroad back home, nobody knows how long the war will go on or how it might escalate or end, and so many Iranians and foreign nationals are escaping Iran if they can (this the border with Pakistan), but many others are trying to get "But many others are trapped where they are, everyone afraid of what could come next." - Caroline Hoarded, BBC News While the world is waiting to see what President Trump will do next (we've not had any reaction from the White House so far, there's been no flurry of social media posts on him like we saw yesterday, although that could happen at any moment), there are reports that the US is leading towards involvement rather than diplomacy, and that would involve sending US troops to the region to use those bunker-buster bombs to destroy Iran's nuclear capabilities deep underground. Make no mistake, if that is what the US does, each This is a profoundly defining moment of Trump's presidency. Seismic. This is a man who has long campaigned for no foreign wars, attacked Barack Obama, accusing him of trying to start a war with Iran, and only back in May said in Saudi Arabia that the era of foreign intervention was over, so lots of his supporters will be saying, "What happened to that man? Could there be regime change?" Well, Donald Trump also said yesterday the threat to potentially kill the supreme leader, so "we just Nomia, thank you. Well, let's get the picture then from Jerusalem. Our correspondent, Hugo Bichega, is there on the ground. Hugo, how will these words have been received and what's the latest on the conflict from that perspective? Yes, you know, so Israel has continued to attack Iran. The military said more than 50 planes hit about twenty targets across Tehran overnight, including weapons production facilities, and on the ground in Iran we're seeing those huge traffic jams of residents trying to flee the city, Tehran, because of fears of an escalation of this conflict, shortages of petrol and food are being filed, also some cash machines The Iranian Okay, Hugo, in Jerusalem, thank you. "Thank you.
"""

# Split text if longer than model's token limit (~1024 tokens)
def chunk_text(text, max_tokens=1024):
    nltk.download('punkt')
    from nltk.tokenize import sent_tokenize

    sentences = sent_tokenize(text)
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_tokens:
            chunk += " " + sentence
        else:
            chunks.append(chunk.strip())
            chunk = sentence
    chunks.append(chunk.strip())
    return chunks

chunks = chunk_text(transcript)

# Summarize each chunk
summaries = []
for i, chunk in enumerate(chunks):
    summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    summaries.append(summary)

# Combine summaries
final_summary = " ".join(summaries)
print("Summary:\n", final_summary)


c:\Users\adabh\Documents\projects\speechbridge\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adabh\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling bac

Summary:
 Ayatollah Ali Khamenei warns of "irreparable consequences" if the U.S. becomes directly involved in Israel's military campaign against Tehran. It is understood that President Trump is considering joining the attacks on Iran's nuclear sites. Overnight missiles were intercepted over Tel Aviv, some limited damage reported, but no more casualties. In Cyprus two repatriation flights were laid on to bring Israeli stranded abroad back home. Nobody knows how long the war will go on or how it might escalate or end, and so many Iranians and foreign nationals are escaping Iran if they can. There are reports that the US is leading towards involvement rather than diplomacy. That would involve sending US troops to the region to use bunker-buster bombs. This is a profoundly defining moment of Trump's presidency, says Caroline Hoarded. Israel has continued to attack Iran. The military said more than 50 planes hit about twenty targets across Tehran overnight. On the ground in Iran we're seein

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import nltk

# Download NLTK tokenizer
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Load T5-base model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Function to chunk text
def chunk_text(text, max_words=500):
    sentences = sent_tokenize(text)
    chunks, chunk = [], ""
    for sentence in sentences:
        if len(chunk.split()) + len(sentence.split()) <= max_words:
            chunk += " " + sentence
        else:
            chunks.append(chunk.strip())
            chunk = sentence
    chunks.append(chunk.strip())
    return chunks

# Function to summarize one chunk
def summarize_t5(text, max_len=120, min_len=30):
    input_text = "summarize: " + text.strip().replace("\n", " ")
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=max_len, min_length=min_len, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# === RUN PIPELINE ===

# Replace with your Whisper transcript
transcript = """
Iran's supreme leader Ayatollah Ali Khamenei has warned Washington of "irreparable consequences" if the U.S. becomes directly involved in Israel's military campaign against Tehran. It is understood that President Trump is considering joining the attacks on Iran's nuclear sites. Here's our diplomatic correspondent Caroline Hawley. These satellite images show a military base in Tabriz in the northwest of the country, here before. Now a defiant response from Ayatollah Ali Khamenei (his words read out by a presenter on state-run TV). Iranian president For civilians in Tehran, these are "terrifying times", "shops shut", "quiet streets" many people are held up at home, hiding, we're told, in basements and garages many others have already fled the capital, among them Nagis Muhammadi, winner of the Nobel Peace Prize, who spent many years in prison for his death The Hiren is still firing at Israel but not at the same rate as before. But not at the same rate as before: overnight missiles were intercepted over Tel Aviv, some limited damage reported, but no more casualties. In Cyprus two repatriation flights were laid on to bring Israeli stranded abroad back home, nobody knows how long the war will go on or how it might escalate or end, and so many Iranians and foreign nationals are escaping Iran if they can (this the border with Pakistan), but many others are trying to get "But many others are trapped where they are, everyone afraid of what could come next." - Caroline Hoarded, BBC News While the world is waiting to see what President Trump will do next (we've not had any reaction from the White House so far, there's been no flurry of social media posts on him like we saw yesterday, although that could happen at any moment), there are reports that the US is leading towards involvement rather than diplomacy, and that would involve sending US troops to the region to use those bunker-buster bombs to destroy Iran's nuclear capabilities deep underground. Make no mistake, if that is what the US does, each This is a profoundly defining moment of Trump's presidency. Seismic. This is a man who has long campaigned for no foreign wars, attacked Barack Obama, accusing him of trying to start a war with Iran, and only back in May said in Saudi Arabia that the era of foreign intervention was over, so lots of his supporters will be saying, "What happened to that man? Could there be regime change?" Well, Donald Trump also said yesterday the threat to potentially kill the supreme leader, so "we just Nomia, thank you. Well, let's get the picture then from Jerusalem. Our correspondent, Hugo Bichega, is there on the ground. Hugo, how will these words have been received and what's the latest on the conflict from that perspective? Yes, you know, so Israel has continued to attack Iran. The military said more than 50 planes hit about twenty targets across Tehran overnight, including weapons production facilities, and on the ground in Iran we're seeing those huge traffic jams of residents trying to flee the city, Tehran, because of fears of an escalation of this conflict, shortages of petrol and food are being filed, also some cash machines The Iranian Okay, Hugo, in Jerusalem, thank you. "Thank you.
"""

chunks = chunk_text(transcript)
summaries = [summarize_t5(chunk) for chunk in chunks]
final_summary = " ".join(summaries)

print("Final Summary:\n", final_summary)


c:\Users\adabh\Documents\projects\speechbridge\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adabh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\adabh\Documents\projects\speechbridge\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adabh\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingfa

Final Summary:
 ayatollah Ali Khamenei warns of "irreparable consequences" if the u.s. joins the attacks on Iran's nuclear sites . the hiren is still firing at Israel but not at the same rate as before . if that is what the u.s. does, each would send troops to destroy Iran's nuclear capabilities . the military says more than 50 planes hit about twenty targets across Tehran overnight . on the ground in Iran we're seeing those huge traffic jams of residents trying to flee the city .


In [2]:
from transformers import pipeline
import nltk

# Download NLTK sentence tokenizer
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

# Load fast summarizer model
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# === Your input: long transcript ===
transcript = """
Iran's supreme leader Ayatollah Ali Khamenei has warned Washington of "irreparable consequences" if the U.S. becomes directly involved in Israel's military campaign against Tehran. It is understood that President Trump is considering joining the attacks on Iran's nuclear sites. Here's our diplomatic correspondent Caroline Hawley. These satellite images show a military base in Tabriz in the northwest of the country, here before. Now a defiant response from Ayatollah Ali Khamenei (his words read out by a presenter on state-run TV). Iranian president For civilians in Tehran, these are "terrifying times", "shops shut", "quiet streets" many people are held up at home, hiding, we're told, in basements and garages many others have already fled the capital, among them Nagis Muhammadi, winner of the Nobel Peace Prize, who spent many years in prison for his death The Hiren is still firing at Israel but not at the same rate as before. But not at the same rate as before: overnight missiles were intercepted over Tel Aviv, some limited damage reported, but no more casualties. In Cyprus two repatriation flights were laid on to bring Israeli stranded abroad back home, nobody knows how long the war will go on or how it might escalate or end, and so many Iranians and foreign nationals are escaping Iran if they can (this the border with Pakistan), but many others are trying to get "But many others are trapped where they are, everyone afraid of what could come next." - Caroline Hoarded, BBC News While the world is waiting to see what President Trump will do next (we've not had any reaction from the White House so far, there's been no flurry of social media posts on him like we saw yesterday, although that could happen at any moment), there are reports that the US is leading towards involvement rather than diplomacy, and that would involve sending US troops to the region to use those bunker-buster bombs to destroy Iran's nuclear capabilities deep underground. Make no mistake, if that is what the US does, each This is a profoundly defining moment of Trump's presidency. Seismic. This is a man who has long campaigned for no foreign wars, attacked Barack Obama, accusing him of trying to start a war with Iran, and only back in May said in Saudi Arabia that the era of foreign intervention was over, so lots of his supporters will be saying, "What happened to that man? Could there be regime change?" Well, Donald Trump also said yesterday the threat to potentially kill the supreme leader, so "we just Nomia, thank you. Well, let's get the picture then from Jerusalem. Our correspondent, Hugo Bichega, is there on the ground. Hugo, how will these words have been received and what's the latest on the conflict from that perspective? Yes, you know, so Israel has continued to attack Iran. The military said more than 50 planes hit about twenty targets across Tehran overnight, including weapons production facilities, and on the ground in Iran we're seeing those huge traffic jams of residents trying to flee the city, Tehran, because of fears of an escalation of this conflict, shortages of petrol and food are being filed, also some cash machines The Iranian Okay, Hugo, in Jerusalem, thank you. "Thank you.
"""

# === Function to split transcript into smaller chunks ===
def chunk_text(text, max_words=300):
    sentences = sent_tokenize(text)
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk.split()) + len(sentence.split()) <= max_words:
            chunk += " " + sentence
        else:
            chunks.append(chunk.strip())
            chunk = sentence
    if chunk:
        chunks.append(chunk.strip())
    return chunks

# === Chunk the transcript ===
chunks = chunk_text(transcript)

# === Generate summary for each chunk ===
summaries = []
for i, chunk in enumerate(chunks):
    print(f"Summarizing chunk {i + 1}/{len(chunks)}...")
    summary = summarizer(
        chunk,
        max_length=100,  # Can reduce this for shorter summaries
        min_length=30,
        do_sample=False
    )[0]["summary_text"]
    summaries.append(summary)

# === Combine summaries ===
final_summary = " ".join(summaries)

# === Output ===
print("\n Final Summary:\n")
print(final_summary)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adabh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\adabh\Documents\projects\speechbridge\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adabh\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-st

Summarizing chunk 1/2...
Summarizing chunk 2/2...

 Final Summary:

 Iran's supreme leader Ayatollah Ali Khamenei warns U.S. of 'irreparable consequences' Iran's Hiren missiles are still firing at Israel but not at the same rate as before . Civilians in Tehran are being held up at home, hiding in basements and garages .  There are reports that the US is leading towards involvement rather than diplomacy, and that would involve sending US troops to the region to use bunker-buster bombs to destroy Iran's nuclear capabilities deep underground . This is a profoundly defining moment of Trump's presidency, says Caroline Hoarded .
